# Transformerモデル（分類タスク用）の実装
クラス分類のTransformerモデルの実装

## 目標
1.	Transformerのモジュール構成を理解する
2.	LSTMやRNNを使用せずCNNベースのTransformerで自然言語処理が可能な理由を理解する
3.	Transformerを実装できるようになる

## Library

In [64]:
import math
import numpy as np
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchtext

In [65]:
# Setup seeds
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

In [66]:
class Embedder(nn.Module):
    """
    idで示されている単語をベクトルに変換する
    """
    
    def __init__(self, text_embedding_vectors):
        super(Embedder, self).__init__()
        
        self.embeddings = nn.Embedding.from_pretrained(
            embeddings=text_embedding_vectors, freeze=True)
        # freeze=Trueによりバックプロパゲーションで更新されず変化しなくなる
        
    def forward(self, x):
        x_vec = self.embeddings(x)
            
        return x_vec

In [68]:
# 動作確認
from utils.dataloader import get_IMDb_DataLoaders_and_TEXT
train_dl, val_dl, test_dl, TEXT = get_IMDb_DataLoaders_and_TEXT(
    max_length=256, batch_size=24)

# ミニバッチ
batch = next(iter(train_ld))

# モデル構築
net1 = Embedder(TEXT.vocab.vectors)

# 入出力
# idで表現されているデータを一つ取ってきてEmbedderに入れてちゃんとベクトル形式で表現されるか確認する
x = batch.Text[0]
x1 = net1(x)

print('入力のテンソルサイズ：', x.shape)
print('出力のテンソルサイズ:', x1.shape)




入力のテンソルサイズ： torch.Size([24, 256])
出力のテンソルサイズ: torch.Size([24, 256, 300])


In [69]:
dir(batch)

['Label',
 'Text',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_get_field_values',
 'batch_size',
 'dataset',
 'fields',
 'fromvars',
 'input_fields',
 'target_fields']

In [70]:
len(batch.Text)  

2

In [71]:
len(batch.Label)  

24

In [72]:
print(batch.Text[0])             # 各文章をidで示したもの
print(batch.Text[0].size())  

tensor([[   2,   14,   55,  ...,    1,    1,    1],
        [   2,  396,    6,  ...,    1,    1,    1],
        [   2,   38, 9369,  ...,   66,   48,    3],
        ...,
        [   2,   58,   14,  ...,    5,  940,    3],
        [   2,  552,    6,  ...,    1,    1,    1],
        [   2, 1595, 9274,  ...,    7,    0,    3]])
torch.Size([24, 256])


In [73]:
print(batch.Text[1])          # 各文章の単語数
print(len(batch.Text[1]))

tensor([163, 154, 256, 154, 195, 256, 256, 256,  66,  94, 256, 256, 159, 177,
        182,  94, 256, 170, 146, 256,  81, 256, 178, 256])
24


In [74]:
batch.Label   # ネガポジのラベル

tensor([0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0])

In [75]:
class PositionalEncoder(nn.Module):
    """
    入力された単語の位置を表すベクトル情報を付加する
    cosとsinを使う
    """
    def __init__(self, d_model=300, max_seq_len=256):
        super().__init__()
        self.d_model = d_model  # 単語ベクトルの次元数
        
        # 単語の順番(pos)と埋め込みベクトルの次元の位置(i)によって一意に定まる値の表をpeとして作成
        pe = torch.zeros(max_seq_len, d_model)
        
        # GPUが使える場合はGPUへ送る、ここでは省略。実際に学習時には使用する
        # device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        # pe = pe.to(device)
        
        for pos in range(max_seq_len):
            for i in range(0, d_model, 2):
                pe[pos, i] = math.sin(pos / (10000**((2*i)/d_model)))
                pe[pos, i+1] = math.cos(pos / (10000**((2*i)/d_model)))
                
        # 表peの先頭にミニバッチ次元となる次元を足す
        self.pe = pe.unsqueeze(0)
        
    def forward(self, x):
        # 入力とpositional encodingを足し算する  掛け算ではない
        # x　がpeよりも小さいので大きくする
        ret = math.sqrt(self.d_model)*x + self.pe  # ブロードキャストされる？
        return ret
    

In [76]:
# 動作確認

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# モデル構築
net1 = Embedder(TEXT.vocab.vectors)
net2 = PositionalEncoder(d_model=300, max_seq_len=256)
# net1.to(device)
# net2.to(device)

# 入出力
x = batch.Text[0]
x1 = net1(x)
x2 = net2(x1)

print('入力のテンソルサイズ：', x1.shape)
print('出力のテンソルサイズ:', x2.shape)

入力のテンソルサイズ： torch.Size([24, 256, 300])
出力のテンソルサイズ: torch.Size([24, 256, 300])


In [77]:
class Attention(nn.Module):
    """
    TransformerはマルチヘッドAttention
    今回はシングルAttentionで実装
    """
    
    def __init__(self, d_model=300):
        super().__init__()
        
        # SAGANではpointwiseだったが、Transformerでは全結合で特徴量を変換する
        self.q_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        
        # 出力時に使用する全結合層
        self.out = nn.Linear(d_model, d_model)
        
        # Attentionの大きさ調節の変数
        self.d_k = d_model
        
    def forward(self, q, k, v, mask):
        # 全結合層で特徴量を変換
        k = self.k_linear(k)
        q = self.q_linear(q)
        v = self.v_linear(v)
        
        # Attentionの値を計算
        # 各値を足し算すると大きくなりすぎるのでroot(d_k)でわって調整
        weights = torch.matmul(q, k.transpose(1,2) / math.sqrt(self.d_k))
        
        # ここでmaskを計算
        mask = mask.unsqueeze(1)
        weights = weights.masked_fill(mask==0, -1e9)
        
        # softmaxで規格化
        # 各単語一つずつ正規化していく
        normlized_weights = F.softmax(weights, dim=-1)
        
        # AttentionをValueと掛け算
        output = torch.matmul(normlized_weights, v)
        
        # 全結合層で特徴量を変換
        output = self.out(output)
    
        return output, normlized_weights

In [78]:
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff=1024, dropout=0.1):
        """
        Attention層から出力を単純に全結合層２つで特徴量を変換するユニット
        重みを増やして表現力を大きくするためと捉えておけばいい？
        """
        super().__init__()
        self.linear_1 = nn.Linear(d_model, d_ff)
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(d_ff, d_model)
        
    def forward(self, x):
        x = self.linear_1(x)
        x = self.dropout(F.relu(x))
        x = self.linear_2(x)
        return x

In [79]:
class TransformerBlock(nn.Module):
    def __init__(self, d_model, dropout=0.1):
        super().__init__()
        
        # layernormalization層
        self.norm_1 = nn.LayerNorm(d_model)
        self.norm_2 = nn.LayerNorm(d_model)
        
        # Attention層
        self.attn = Attention(d_model)
        
        # Attention層の後の全結合層２つ
        self.ff = FeedForward(d_model)
        
        # Dropout
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)
        
    def forward(self, x, mask):
        # 正規化とAttention
        x_normlized = self.norm_1(x)
        output, normlized_weights = self.attn(x_normlized, x_normlized, x_normlized, mask)
        
        x2 = x+self.dropout_1(output)
        
        # 正規化と全結合層
        x_normlized2 = self.norm_2(x2)
        output = x + self.dropout_2(self.ff(x_normlized2))
        
        return output, normlized_weights

In [80]:
# 動作確認

# モデル構築
net1 = Embedder(TEXT.vocab.vectors)
net2 = PositionalEncoder(d_model=300, max_seq_len=256)
net3 = TransformerBlock(d_model=300)

# mask作成
x = batch.Text[0][1]
input_pad = 1  # <pad>のidは１
input_mask = (x!=input_pad)
print(input_mask[0])

# 入出力
x = batch.Text[0]
x1 = net1(x)     # 単語をベクトルにする
x2 = net2(x1)  # Position情報を加算
x3, normlized_weights = net3(x2, input_mask)   # self-attentinoで特徴量を変換

print('入力のテンソルサイズ：', x2.shape)
print('出力のテンソルサイズ:', x3.shape)
print('Attentionのサイズ:', normlized_weights.shape)

tensor(True)
入力のテンソルサイズ： torch.Size([24, 256, 300])
出力のテンソルサイズ: torch.Size([24, 256, 300])
Attentionのサイズ: torch.Size([24, 256, 256])


In [81]:
batch.Text[1]

tensor([163, 154, 256, 154, 195, 256, 256, 256,  66,  94, 256, 256, 159, 177,
        182,  94, 256, 170, 146, 256,  81, 256, 178, 256])

In [82]:
class ClassificationHead(nn.Module):
    """
    Transformer_Blockの出力を使用し、最後にクラス分類する
    """
    
    def __init__(self, d_model=300, output_dim=2):
        super().__init__()
        
        # 全結合層
        self.linear = nn.Linear(d_model, output_dim)  # neg, posi の２値分類
        
        # 重み初期化処理
        nn.init.normal_(self.linear.weight, std=0.02)
        nn.init.normal_(self.linear.bias, 0)
        
    def forward(self, x):
        x0 = x[:, 0, :]  # 各バッチの各文の先頭の単語<cls>の特徴量(３００次元)を取り出す
        # 最初は各バッチの各文の先頭の単語<cls>にネガポジに重要な特徴量が集まるわけではないが
        # これを使って損失関数から学習させることでネガポジに重要な特徴量が集まるようになる
        out = self.linear(x0)
        
        return out

In [83]:
# 動作確認

# ミニバッチの用意
batch = next(iter(train_dl))

# モデル構築
net1 = Embedder(TEXT.vocab.vectors)
net2 = PositionalEncoder(d_model=300, max_seq_len=256)
net3 = TransformerBlock(d_model=300)
net4 = ClassificationHead(output_dim=2, d_model=300)

# 入出力
x = batch.Text[0]
x1 = net1(x)  # 単語をベクトルに
x2 = net2(x1)  # Positon情報を足し算
x3, normlized_weights = net3(x2, input_mask)  # Self-Attentionで特徴量を変換
x4 = net4(x3)  # 最終出力の0単語目を使用して、分類0-1のスカラーを出力

print("入力のテンソルサイズ：", x3.shape)
print("出力のテンソルサイズ：", x4.shape)  # 2値になった！

入力のテンソルサイズ： torch.Size([24, 256, 300])
出力のテンソルサイズ： torch.Size([24, 2])


In [86]:
# 最終的なTransformerモデル
class TransformerClassification(nn.Module):
    """
    Transformerでクラス分類させる
    """
    
    def __init__(self, text_embedding_vectors, d_model=300, max_seq_len=256, output_dim=2):
        super().__init__()
        
        # モデル構築
        self.net1 = Embedder(text_embedding_vectors)
        self.net2 = PositionalEncoder(d_model=d_model, max_seq_len=max_seq_len)
        self.net3_1 = TransformerBlock(d_model=d_model)
        self.net3_2 = TransformerBlock(d_model=d_model)
        self.net4 = ClassificationHead(output_dim=output_dim, d_model=d_model)
        
    def forward(self, x, mask):
        x1 = self.net1(x)
        x2 = self.net2(x1)
        x3_1, normlized_weights_1 = self.net3_1(x2, mask)
        x3_2, normlized_weights_2 = self.net3_2(x3_1, mask)
        x4 = self.net4(x3_2)
        return x4, normlized_weights_1, normlized_weights_2

In [87]:
# 動作確認

# ミニバッチの用意
batch = next(iter(train_dl))

# モデル構築
net = TransformerClassification(
    text_embedding_vectors=TEXT.vocab.vectors, d_model=300, max_seq_len=256, output_dim=2)

# 入出力
x = batch.Text[0]
input_mask = (x != input_pad)
out, normlized_weights_1, normlized_weights_2 = net(x, input_mask)

print("出力のテンソルサイズ：", out.shape)
print("出力テンソルのsigmoid：", F.softmax(out, dim=1))
# これは何も学習していない状態でmodelにデータを流し込んだもの

出力のテンソルサイズ： torch.Size([24, 2])
出力テンソルのsigmoid： tensor([[0.7040, 0.2960],
        [0.7210, 0.2790],
        [0.7165, 0.2835],
        [0.7176, 0.2824],
        [0.7153, 0.2847],
        [0.6998, 0.3002],
        [0.7044, 0.2956],
        [0.6537, 0.3463],
        [0.7234, 0.2766],
        [0.7117, 0.2883],
        [0.7344, 0.2656],
        [0.7086, 0.2914],
        [0.7084, 0.2916],
        [0.6949, 0.3051],
        [0.7348, 0.2652],
        [0.7108, 0.2892],
        [0.7294, 0.2706],
        [0.7114, 0.2886],
        [0.6899, 0.3101],
        [0.7187, 0.2813],
        [0.7341, 0.2659],
        [0.7272, 0.2728],
        [0.6806, 0.3194],
        [0.7194, 0.2806]], grad_fn=<SoftmaxBackward>)


In [89]:
out

tensor([[1.8202, 0.9536],
        [1.8337, 0.8841],
        [1.8335, 0.9062],
        [1.7762, 0.8435],
        [1.7039, 0.7828],
        [1.7933, 0.9470],
        [1.8516, 0.9832],
        [1.6864, 1.0509],
        [1.8136, 0.8524],
        [1.8807, 0.9773],
        [1.9339, 0.9167],
        [1.8253, 0.9366],
        [1.7886, 0.9010],
        [1.8111, 0.9879],
        [1.8514, 0.8325],
        [1.8015, 0.9021],
        [1.8377, 0.8462],
        [1.7442, 0.8418],
        [1.6814, 0.8817],
        [1.7247, 0.7865],
        [1.8430, 0.8273],
        [1.8306, 0.8499],
        [1.7425, 0.9862],
        [1.8266, 0.8851]], grad_fn=<AddmmBackward>)